In [1]:
import numpy as np
import pandas as pd

In [2]:
import pickle

In [3]:
import catboost

In [4]:
from ipynb.fs.full import process_functions as func

In [5]:
import importlib

In [6]:
importlib.reload(func)

<module 'ipynb.fs.full.process_functions' (D:\Learning\GeekBrains\_Final Project\process_functions.ipynb)>

In [7]:
filename = 'preliminary_trained_model.pkl'

In [8]:
model = pickle.load(open(filename, 'rb'))

In [9]:
valid_df = pd.read_csv('project_data/attempt_4/test.csv')

In [10]:
#df_date = pd.read_pickle('macroeconomic.pkl')

In [11]:
#df_complete = valid_df.merge(df_date, how='left', left_on='timestamp', right_on='timestamp')

In [12]:
df_complete = valid_df.fillna(0)

In [13]:
X_test = func.process(valid_df, 'test')

In [14]:
X_test.head()

,full_sq,life_sq,floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,39.0,20.700000,2,1,1998.0,1,8.9,3,Investment,Juzhnoe Butovo,...,8,0,0,0,1,10,1,0,14,1
1,79.2,34.126118,8,1,0.0,3,1.0,1,OwnerOccupier,Poselenie Vnukovskoe,...,4,1,1,0,2,11,0,1,12,1
2,40.5,25.100000,3,2,1960.0,2,4.8,2,Investment,Perovo,...,42,11,4,0,10,21,0,10,71,11
3,62.8,36.000000,17,1,2016.0,2,62.8,3,OwnerOccupier,Poselenie Voskresenskoe,...,1,1,2,0,0,10,0,0,2,0
4,40.0,40.000000,17,1,0.0,1,1.0,1,OwnerOccupier,Poselenie Vnukovskoe,...,5,1,1,0,2,12,0,1,11,1


In [15]:
X_test.shape

(7662, 288)

In [16]:
    cat_columns = ['product_type', 'sub_area', 'culture_objects_top_25', 'thermal_power_plant_raion', 
                   'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 
                   'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 
                   'big_road1_1line', 'railroad_1line', 'ecology']

In [17]:
X_test.state.dtype

dtype('int64')

In [18]:
X_test.isna().sum()

full_sq               0
life_sq               0
floor                 0
material              0
build_year            0
                     ..
church_count_5000     0
mosque_count_5000     0
leisure_count_5000    0
sport_count_5000      0
market_count_5000     0
Length: 288, dtype: int64

In [19]:
X_test[cat_columns]  =X_test[cat_columns].astype('category')

In [20]:
submissions = model.predict(X_test)

CatBoostError: Invalid type for cat_feature[object_idx=4662,feature_idx=8]=NaN : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [60]:
submissions

array([ 5924858.05309915,  9916951.06138354,  4892607.41267314, ...,
       -1308363.44179923,  4406463.55423659, 12912744.28472111])

In [61]:
result = pd.DataFrame(valid_df['id'])

In [62]:
result['price_doc_0'] = submissions.tolist()

In [63]:
result = result.set_index('id')

In [64]:
result.head()

,price_doc_0
id,
30474,5.924858e+06
30475,9.916951e+06
30476,4.892607e+06
30477,8.939420e+06
30478,4.757160e+06


In [65]:
result['price_doc'] = result['price_doc_0'].apply(lambda x: round(abs(x), 2) if x < 0 else round(x, 2))

In [66]:
result.drop('price_doc_0', axis = 1, inplace=True)

In [67]:
result.to_csv('submission.csv')

In [68]:
result.shape

(7662, 1)